In [0]:
import mlflow
import torch
import pandas as pd
import numpy as np
import cloudpickle

In [0]:
registered_model_name = "bert4rec_v4"
model_uri = f"models:/{registered_model_name}/3" # 버전은 상황에 맞게 조정
loaded_model = mlflow.pyfunc.load_model(model_uri)

In [0]:
# 추론할 데이터 (Pandas DataFrame 형태)
inference_data = pd.DataFrame({
    "movie_history": [
        #[1, 89745, 4993, 5952, 7153],
        [4896,1,4993,5952,33794,91529,109487,8368,7153,89745,5816,58559]
    ]
})

# 예측 수행
recommendations = loaded_model.predict(inference_data)
print("\n--- MLflow 모델 추론 결과 ---")
print(recommendations)

In [0]:
result_movieids = recommendations.iloc[0].tolist()[0]
result_movieids

In [0]:
def get_movie_titles(movie_ids, movies_table_name):
    """
    주어진 movie_ids 리스트에 해당하는 영화 제목을 반환합니다.

    Args:
        movie_ids (list): 영화 ID 리스트
        movies_table_name (str): 영화 메타데이터가 있는 Spark 테이블 이름

    Returns:
        list: 영화 제목 리스트
    """
    query = f"""
    SELECT title
    FROM {movies_table_name}
    WHERE movieId IN ({','.join(map(str, movie_ids))})
    """
    titles_df = spark.sql(query)
    titles_list = titles_df.select("title").rdd.flatMap(lambda x: x).collect()
    return titles_list

In [0]:
# 예시 사용법
movie_ids = result_movieids
movies_table_name = "1dt_team8_databricks.`movielens-32m`.movies"
titles = [{'title': title} for title in get_movie_titles(movie_ids, movies_table_name)]
display(titles)